## Modify the Model

### Retrain last layer's linear model

The original VGG16 network's last layer is Dense (a linear model), so it is a little odd and wasterful that we are adding an additional linear model on top of it in lesson 2. 

Also, you may notice that the last layer had a softmax activation, which is an odd choice for an intermediate layer after we add another linear layer to a model.

So, we start by removing the last layer, and telling Keras to fix the weights in all the other layers.

In [2]:
%matplotlib inline
from importlib import reload
import utils; reload(utils)
from utils import *

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

ModuleNotFoundError: No module named 'matplotlib'

In [1]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

vgg.model.summary()

Using TensorFlow backend.
/Users/yingchipei/Documents/github/fastai-notes/deeplearning1/nbs/vgg16.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
/Users/yingchipei/Documents/github/fastai-notes/deeplearning1/nbs/vgg16.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
/Users/yingchipei/Documents/github/fastai-notes/deeplearning1/nbs/vgg16.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
/Users/yingchipei/Documents/github/fastai-notes/deeplearning1/nbs/vgg16.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [2]:
model.pop()  # remove the last layer
for layer in model.layers:
    layer.trainable = False

**WARNING:** Now that we have modified the definition of _model_, be careful not to rerun any code in the previous sections.

In [3]:
model.add(Dense(2, activation = 'softmax'))

NameError: name 'Dense' is not defined